In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Multifonte Spark") \
    .config(
        "spark.jars.packages",
        ",".join([
            "org.postgresql:postgresql:42.6.0",
            "org.apache.hadoop:hadoop-aws:3.3.4"
        ])
    ) \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

In [2]:
caminho = "s3a://dados/"

In [3]:
df = spark.readStream.format("csv") \
         .option("header", "true") \
         .schema("nome STRING, idade INT") \
         .load(caminho)

In [4]:
contagem = df.groupBy("nome").count()
    

In [6]:
def salvar_no_postgres(batch_df,epoch_id):
        batch_df.write \
             .format("jdbc") \
             .option("url", "jdbc:postgresql://postgres_streaming:5432/streaming") \
             .option("dbtable", "resultados") \
             .option("user", "spark") \
            .option("password", "spark123") \
             .option("driver", "org.postgresql.Driver") \
            .mode("append") \
            .save()

In [7]:
contagem.writeStream \
    .outputMode("complete") \
    .foreachBatch(salvar_no_postgres) \
    .start()